In [1]:
import os
import sys

import numpy as np
#from skimage import io
#import skimage
import matplotlib.pyplot as plt
#from meta_load import *

sys.path.append(os.path.join("..",'Data_prep'))
sys.path.append(os.path.join("..",'Models2'))

from Custom_dataloader import *
import pandas as pd
import matplotlib.pyplot as plt
import pickle

DB="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=aorus_2,volume=Paltas_DataBase"
Dbd="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=aorus_2,volume=Paltas_DataBase/Data_Base"
#DBd="//MYCLOUDPR4100/Paltas_DataBase/Data_Base"
#DB="//MYCLOUDPR4100/Paltas_DataBase"
#d_t=transforms.Compose([phantom_segmentation(False)])

datar=Dataset_direct(root_dir=Dbd,ImType=['PhantomRGB'],Intersec=False,retrieve_img=False)

pd.options.display.max_columns = None

In [21]:
os.listdir(os.path.join(DB,"metadata"))
meta=['bl_per_wmask_phantom','stat_val_hsv_wmask_phantom','stat_val_lab_wmask_phantom']
ddlist=np.array(os.listdir(os.path.join(DB,"metadata",meta[0])))
dlist=np.vectorize(lambda d:d.split(".")[0])(ddlist)

In [ ]:
date=np.vectorize(lambda d:("_").join(datar[np.where(datar.aID==d)[0][0]]["Date"].split("_")[:2]))(dlist)
Y=np.vectorize(lambda d:datar[np.where(datar.aID==d)[0][0]]["landmarks"])(dlist)
timei=np.vectorize(lambda d:datar[np.where(datar.aID==d)[0][0]]['PhantomRGB_metadata']['DateTime'])(dlist)

In [ ]:
dat=np.vectorize(lambda d:(pd.read_pickle(os.path.join(DB,"metadata",meta[0],d)))[d.split('.')[0]])(ddlist).reshape(-1,1)
camp=np.array(['mean','std','mode'])
dat1=np.vectorize(lambda d,c:(pd.read_pickle(os.path.join(DB,"metadata",meta[1],d)))[d.split('.')[0]][c],
            signature="(),()->()")(ddlist.reshape(-1,1),camp)
dat2=np.vectorize(lambda d,c:(pd.read_pickle(os.path.join(DB,"metadata",meta[2],d)))[d.split('.')[0]][c],
            signature="(),()->()")(ddlist.reshape(-1,1),camp)


In [ ]:
date=np.vectorize(lambda d:("_").join(datar[np.where(datar.aID==d)[0][0]]["Date"].split("_")[:2]))(dlist)

# VAE GEN

In [2]:
meta_dir="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=aorus_2,volume=Paltas_DataBase/metadata_VAE_v2"
metad=np.array(os.listdir(meta_dir)[1:])
metad_dict=np.vectorize(lambda meta: pickle.load(open(os.path.join(meta_dir,meta),'rb')))(metad)
meta_u=np.vectorize(lambda dic:dic['u'],signature='()->(j)')(metad_dict)
meta_sig=np.vectorize(lambda dic:dic['sig'],signature='()->(j)')(metad_dict)
meta_C=np.vectorize(lambda dic:dic['Class'][0],otypes=[object],signature='()->()')(metad_dict)
meta_D=np.vectorize(lambda dic:dic['Date'][0],otypes=[object],signature='()->()')(metad_dict)

In [13]:
dfc = ['meta_u_'+str(i) for i in range(meta_u.shape[1])]
dfc.extend(['meta_sig_'+str(i) for i in range(meta_sig.shape[1])])
dfc.extend(['Class', 'Date'])
df_vae = pd.DataFrame(data = np.concatenate((meta_u, meta_sig, meta_C.reshape(-1,1), meta_D.reshape(-1,1)), axis = 1), columns= dfc)
df_vae

,meta_u_0,meta_u_1,meta_u_2,meta_u_3,meta_u_4,meta_u_5,meta_u_6,meta_u_7,meta_u_8,meta_u_9,meta_u_10,meta_u_11,meta_u_12,meta_u_13,meta_u_14,meta_u_15,meta_u_16,meta_u_17,meta_u_18,meta_u_19,meta_sig_0,meta_sig_1,meta_sig_2,meta_sig_3,meta_sig_4,meta_sig_5,meta_sig_6,meta_sig_7,meta_sig_8,meta_sig_9,meta_sig_10,meta_sig_11,meta_sig_12,meta_sig_13,meta_sig_14,meta_sig_15,meta_sig_16,meta_sig_17,meta_sig_18,meta_sig_19,Class,Date
0,0.078699,0,0.142968,0.291437,0.194801,0.252386,0.280699,0.194773,0,0,0.172331,0,0.0494865,0.0215562,0.728413,0.137939,0.348239,0.356934,0,0.135583,0,0.302503,0,0.43552,0.163887,0.414817,0,0.465426,0.442327,0.522759,0.596871,0.174845,0.00303755,0,0.586983,0,0.354402,0.142688,0.515514,0.648625,N_Control,11_junio_1
1,0.60877,0,0,1.31731,0.859336,0.470032,0.406602,0.933932,0.55057,0,1.10621,0.161511,0,0.368912,0.0980214,0,0.310254,0.825673,0,0.912154,0,0.153436,0,0.389453,0.51758,0.974635,0,0.326291,0.207235,0.585391,1.314,0.208379,0,0,0.85189,0.771395,0.31974,0,1.00241,0.462932,P_Control,11_junio_1
2,0.686334,0,0,1.27835,0.952391,0.331739,0.410137,0.94442,0.512473,0,0.974884,0.155276,0,0.374923,0.0433354,0,0.361175,0.909057,0,0.994359,0,0.177162,0,0.373624,0.472896,0.911373,0,0.417087,0.317145,0.564312,1.15435,0.257537,0,0,0.75731,1.08979,0.162783,0,0.765707,0.417435,K_Control,11_junio_1
3,0.844607,0,0,1.48797,1.04294,0.0569124,0.388496,1.28495,0.94787,0,0.779879,0.776381,0,0.213397,0,0,0.418255,0.964953,0,1.15482,0,0,0,0.399854,1.0511,0.997991,0,0.269001,0.0784226,0.700827,1.45894,0.365634,0,0.480788,0.872483,1.19107,0.122678,0,1.06638,0.149918,Control,11_junio_1
4,0.137993,0,0,0.649189,0.256893,0.472469,0.317355,0.611884,0.29892,0,0.294118,0,0,0.155024,0.566162,0,0.122542,0.440422,0,0.39332,0,0.179885,0,0.381679,0.256927,0.562251,0,0.297674,0.169148,0.50813,0.693811,0.232407,0,0,0.522512,0.0953433,0.242455,0.112508,0.815429,0.370586,H50%,11_junio_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3848,0.28493,0.730358,0.496297,0,0,0.418072,0.223871,0.110502,0,0.545941,0.13817,0,0.67614,0.310144,0.324865,0.677347,0.390191,0.214423,0.53383,0,0,0.520943,0,0.363231,0,0.31814,0,0.549257,0.127501,0.447453,0.0454063,0.178778,0.468734,0,0.572904,0,0.221457,0.347038,0.202071,0.592896,K_Control,9_julio_1
3849,0.257077,0,0,0.385557,0.23176,0.276198,0.29632,0.355897,0,0,0,0,0.0732139,0.28656,0.556216,0.0549994,0.400198,0.39808,0,0.263994,0,0.313272,0,0.299594,0,0.51539,0,0.541148,0.14163,0.549793,0.356229,0.207649,0.257204,0,0.646378,0.271761,0.163825,0.25231,0.610014,0.497658,Control,9_julio_1
3850,0.244874,0.5573,0.632031,0,0,0.442473,0,0,0,1.094,0.422936,0,0.680724,0.735275,0,0.992674,0.139912,0,0.652989,0,0.641461,0.422667,0.785014,0.241689,0,0,0.847837,0.609156,0.265621,0.244972,0,0.0423363,0.69055,0,0.191527,0,0.196538,0.611789,0,1.0747,H50%,9_julio_1
3851,0.410709,0.0607017,0.0858512,0.432054,0.190134,0.301681,0.228655,0.293132,0,0,0.0981167,0,0.0736585,0.0807283,0.544911,0.211439,0.438692,0.360667,0.0727972,0.182478,0,0.301238,0,0.184406,0,0.684036,0,0.392276,0.0121691,0.634411,0.569025,0,0.0749826,0,0.798612,0.717082,0.00302238,0.0495127,0.452348,0.603918,H75%,9_julio_1


In [14]:
df_vae.to_csv('/home/liarpi2/Escritorio/metadata_VAE_v2.csv')

# REG

In [ ]:
datafd={
    'ID':dlist,
    'lab_mean':dat2[:,0],
    'lab_std':dat2[:,1],
    'lab_mode':dat2[:,2],
    'hsv_mean':dat1[:,0],
    'hsv_std':dat1[:,1],
    'hsv_mode':dat1[:,2],
    'black_per':dat.reshape(-1,),
    'hour':np.vectorize(lambda cdt:cdt.split(' ')[1].split(':')[0])(timei),
    'date':date,
    'Y':Y
}
data=pd.DataFrame(datafd)
data

In [ ]:
data.to_csv('/home/liarpi2/Escritorio/metadata_VAE_v2.csv')

In [ ]:
print(datar[0]['PhantomRGB_metadata']['DateTime'])
print(datar[5]['PhantomRGB_metadata']['DateTime'])

In [ ]:
#Normalize
import sklearn
from sklearn.preprocessing import StandardScaler
cont=['lab_mean','lab_std','lab_mode','hsv_mean','hsv_std','hsv_mode','black_per']
features = data[cont]


scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
data[cont]=features
data

In [ ]:
#Encode time and day
data=pd.concat([data,pd.get_dummies(data['hour'],columns='hour'),pd.get_dummies(data['date'],columns='date')],axis=1)
data

In [ ]:
hour_cl=np.unique(np.array(data['hour']))
print(hour_cl)
day_cl=np.unique(np.array(data['date']))
print(day_cl)
data.columns

In [ ]:
#Clasificación para detección de deficiencia hidrica
data=data[(data['Y']=='Control')|\
          #(data['Y']=='K_Control')|\
          #(data['Y']=='N_Control')|\
          #(data['Y']=='P_Control')|\
          (data['Y']=='H50%')\
           #(data['Y']=='H75%')\
         ]
#X=data[list(data.columns[1:7]) + list(data.columns[16:])]
X=data[list(data.columns[1:7])]
print(X.columns)
Yo=((data['Y']=='Control') | (data['Y']=='K_Control') | (data['Y']=='N_Control') | (data['Y']=='P_Control')).astype(int)
Yo


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

GR=np.arange(1,6)
def LR_GR(GR,X,Y):
    polynomial_features = PolynomialFeatures(degree=GR)
    r=polynomial_features.fit_transform(X[:,:6])
    Xco=np.hstack((r,np.array(X[:,6:])))
    X_train, X_test, y_train, y_test = train_test_split(Xco, Y, random_state=20,train_size=0.8)
    Lr1=LogisticRegression(max_iter=4000,C=5)
    Lr1.fit(X_train,y_train)
    pr_tr=Lr1.score(X_train,y_train)
    pr_ts=Lr1.score(X_test,y_test)
    print(GR)
    print(' R^2_train: ', pr_tr)
    print(' R^2_test: ', pr_ts)
    return Lr1,GR,pr_tr,pr_ts
v_LR_GR=np.vectorize(LR_GR,signature='(),(j,k),(l)->(),(),(),()')
LR,GR,pr_tr2,pr_ts2=v_LR_GR(GR,X,np.array(Yo))

plt.plot(GR,pr_tr2, 'ro--')
plt.plot(GR,pr_ts2, 'bo--')
plt.show()

plt.bar(np.arange(LR[0].coef_[0].shape[0]),np.abs(LR[0].coef_[0]))
plt.show()

plt.bar(np.arange(LR[0].coef_[0].shape[0]),np.abs(LR[0].coef_[0].sort()))
plt.show()

In [ ]:
polynomial_features = PolynomialFeatures(degree=1)
Xc=polynomial_features.fit_transform(X[X.columns[:6]])
Xc=np.hstack((X[X.columns[6:]],Xc))
C=np.arange(0.001,3,0.1)
def LR_C(C,X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=20,train_size=0.8)
    Lr1=LogisticRegression(max_iter=4000,C=C)
    Lr1.fit(X_train,y_train)
    pr_tr=Lr1.score(X_train,y_train)
    pr_ts=Lr1.score(X_test,y_test)
    print(C)
    print(' R^2_train: ', pr_tr)
    print(' R^2_test: ', pr_ts)
    return Lr1,C,pr_tr,pr_ts
v_LR_C=np.vectorize(LR_C,signature='(),(j,k),(l)->(),(),(),()')

LR,C,pr_tr1,pr_ts1=v_LR_C(C,Xc,np.array(Yo))

plt.plot(C,pr_tr1, 'ro--')
plt.plot(C,pr_ts1, 'bo--')
plt.show()

In [ ]:
C[np.argmax(pr_ts1)]

In [ ]:
LR[0]

In [ ]:
#np.hstack((np.array(polynomial_features.get_feature_names(X.columns[:6])),np.array(X.columns[6:])))[np.abs(LR[0].coef_[0])>0.2]
np.hstack((np.array(polynomial_features.get_feature_names(X.columns[:6]))))[np.abs(LR[0].coef_[0])>0.1]

In [15]:
from sklearn.tree import DecisionTreeClassifier

DP=np.arange(4,30)
mSL=np.arange(1,70,2)
MSL=np.arange(2,25,1)

#VARIANDO DP
def DT_DP(DP,X,Y):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=20,train_size=0.8)
  Lr1 = DecisionTreeClassifier(max_depth=DP,min_samples_leaf=21,max_leaf_nodes=6)#-----------------------------------------
  #Lr1 = DecisionTreeClassifier(max_depth=DP,min_samples_leaf=8)
  Lr1.fit(X_train,y_train)
  pr_tr=Lr1.score(X_train,y_train)
  pr_ts=Lr1.score(X_test,y_test)
  print(DP)
  print(' R^2_train: ', pr_tr)
  print(' R^2_test: ', pr_ts)
  return Lr1,DP,pr_tr,pr_ts
v_DT_DP=np.vectorize(DT_DP,signature='(),(j,k),(l)->(),(),(),()')

DT1,DP,pr_tr1,pr_ts1=v_DT_DP(DP,X,np.array(Yo))

#VARIANDO mSL
def DT_mSL(mSL,X,Y):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=20,train_size=0.8)
  Lr1 = DecisionTreeClassifier(max_depth=5,min_samples_leaf=mSL,max_leaf_nodes=6)#-----------------------------------------
  #Lr1 = DecisionTreeClassifier(min_samples_leaf=mSL)
  Lr1.fit(X_train,y_train)
  pr_tr=Lr1.score(X_train,y_train)
  pr_ts=Lr1.score(X_test,y_test)
  print(mSL)
  print(' R^2_train: ', pr_tr)
  print(' R^2_test: ', pr_ts)
  return Lr1,mSL,pr_tr,pr_ts
v_DT_mSL=np.vectorize(DT_mSL,signature='(),(j,k),(l)->(),(),(),()')

DT2,mSL,pr_tr2,pr_ts2=v_DT_mSL(mSL,X,np.array(Yo))

#VARIANDO MSL
def DT_MSL(MSL,X,Y):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=20,train_size=0.8)
  Lr1 = DecisionTreeClassifier(max_depth=5,min_samples_leaf=21,max_leaf_nodes=MSL)#-----------------------------------------
  #Lr1 = DecisionTreeClassifier(max_leaf_nodes=MSL,min_samples_leaf=8)
  Lr1.fit(X_train,y_train)
  pr_tr=Lr1.score(X_train,y_train)
  pr_ts=Lr1.score(X_test,y_test)
  print(MSL)
  print(' R^2_train: ', pr_tr)
  print(' R^2_test: ', pr_ts)
  return Lr1,MSL,pr_tr,pr_ts
v_DT_MSL=np.vectorize(DT_MSL,signature='(),(j,k),(l)->(),(),(),()')

DT3,MSL,pr_tr3,pr_ts3=v_DT_MSL(MSL,X,np.array(Yo))



plt.plot(DP,pr_tr1, 'ro--')
plt.plot(DP,pr_ts1, 'bo--')
plt.show()

plt.plot(mSL,pr_tr2, 'ro--')
plt.plot(mSL,pr_ts2, 'bo--')
plt.show()

plt.plot(MSL,pr_tr3, 'ro--')
plt.plot(MSL,pr_ts3, 'bo--')
plt.show()

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
DT3[5]

In [16]:
sklearn.tree.plot_tree(DT3[6],feature_names=X.columns)

NameError: name 'sklearn' is not defined

In [ ]:
df=load_meta_v2()
df